[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [26]:
# import modules

import requests
import pandas as pd
from functools import reduce

from tqdm.auto import tqdm

# 1. Сбор информации о всех строющихся объектах на сайте "наш.дом.рф"


При получении запроса и преобразовании его в json, по ключу "total" можно получить общее кол-во объектов:

In [51]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?sortField=objId&sortType=asc&objStatus=0'
res = requests.get(url)
total_objects = res.json().get('data').get('total')
print(f"Всего объектов: {total_objects}")

Всего объектов: 10615


In [52]:
res.json().get('data')

{'list': [{'objId': 1259,
   'developer': {'devId': 773,
    'shortName': 'КРИСТАЛЛ',
    'fullName': 'КРИСТАЛЛ',
    'groupName': 'Капитал',
    'orgForm': {'id': 4,
     'fullForm': 'Общество с ограниченной ответственностью',
     'shortForm': 'ООО'},
    'companyGroup': 5610,
    'devInn': '6215031345'},
   'rpdRegionCd': 62,
   'objAddr': 'г Рязань, улица Бугровка, д. 0',
   'objCommercNm': '"СМАРТ дом на Новоселов"',
   'objFloorMin': 27,
   'objFloorMax': 27,
   'objElemLivingCnt': 440,
   'objReady100PercDt': '2023-12-31',
   'objSquareLiving': 23985.84,
   'objCompensFundFlg': 0,
   'objProblemFlg': 0,
   'latitude': 54.6045,
   'longitude': 39.8214,
   'problemFlag': 'NONE',
   'siteStatus': 'Строящиеся',
   'objGuarantyEscrowFlg': 1,
   'residentialBuildings': 1,
   'rpdNum': '62-000075',
   'pdId': 1320,
   'buildType': 'Жилое',
   'objStatus': 0},
  {'objId': 1284,
   'developer': {'devId': 175,
    'shortName': 'РОЖДЕСТВЕНО',
    'fullName': 'РОЖДЕСТВЕНО',
    'groupName':

In [30]:
# достанем все objId
limit_ = 100
num_responses = (total_objects + limit_ - 1) // limit_  # готовая формула, вместо условия на остаток от деления
obj_Ids = []

for i in tqdm(range(num_responses)):
    offset = i * limit_
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset}&limit={limit_}&sortField=objId&sortType=asc&objStatus=0'

    res = requests.get(url)
    obj_data = res.json().get('data').get('list')

    obj_cur_ids = [x.get('objId') for x in obj_data]
    obj_Ids.append(obj_cur_ids)

obj_Ids = reduce(lambda x, y: x + y, obj_Ids)

  0%|          | 0/107 [00:00<?, ?it/s]

In [32]:
print(f"Итоговое кол-во выгруженных objId: {len(set(obj_Ids))}")

Итоговое кол-во выгруженных objId: 10615


In [46]:
# выгрузка данных по каждому objId
building_dataset = []
cnt_errors = 0

for objId in tqdm(obj_Ids):
    try:
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{objId}'
        obj_data = requests.get(url).json().get('data')
    except:
        # если указывать конкретную ошибку, то он её почему-то не ловит
        cnt_errors += 1
    else:
        building_dataset.append(obj_data)

  0%|          | 0/10615 [00:00<?, ?it/s]

In [50]:
print(f"Ошибок во время загрузки данных: {cnt_errors}\n"
      f"Всего удалось скачать данных: {len(building_dataset)} из {total_objects}")

Ошибок во время загрузки данных: 3
Всего удалось скачать данных: 10612 из 10615


# 2. Сохранение информации в pandas dataFrame

In [54]:
dataset = pd.json_normalize(building_dataset)

In [56]:
dataset.to_pickle('houses_dataset')